In [1]:
#importing libraries

import warnings
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import gridspec
import plotly.express as px
from pandas import plotting
import missingno as ms
from sklearn.impute import SimpleImputer
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn.preprocessing import OrdinalEncoder
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler
from itertools import product
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer 
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, BaggingClassifier, StackingClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, roc_auc_score, recall_score, precision_score, f1_score
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_excel('Bank_01.xlsb')

In [3]:
df.shape

(715655, 157)

In [4]:
missing_values = df.isna()  
missing_count = missing_values.sum()
percentage_missing = (missing_count / len(df)) * 100
missing_info = pd.DataFrame({'Missing Values': missing_count, 'Percentage Missing': percentage_missing})

print(missing_info) 

                       Missing Values  Percentage Missing
Id                                  0            0.000000
AppsFlyer ID1                  419383           58.601281
Client Code1                   112820           15.764579
Email                          105137           14.691017
Last Name                           0            0.000000
...                               ...                 ...
Device Model                   452320           63.203639
UI                                  1            0.000140
Lead Sub Status Final               0            0.000000
Lead Stage                          0            0.000000
bank_otput                          0            0.000000

[157 rows x 2 columns]


In [5]:
# Count unique values in each column
unique_value_counts = df.nunique()

# Display the counts
print(unique_value_counts)

Id                       218522
AppsFlyer ID1            290487
Client Code1             602426
Email                    610099
Last Name                 94403
                          ...  
Device Model               2065
UI                       715654
Lead Sub Status Final        40
Lead Stage                    6
bank_otput                    2
Length: 157, dtype: int64


In [6]:
!pip install --upgrade pandas

In [8]:
missing_info.to_csv('missing_bank.csv')
unique_value_counts.to_csv('unique_bank.csv')

In [6]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


import pandas as pd
import numpy as np


# In[ ]:


# Function to calculate IV for discrete and conitnous variables on a binary target variable ("Good"/"Bad")


# In[ ]:


def calc_iv_num(df2,feature, target):
    df=df2.copy()
    print(feature)
    lst = []
    df[feature] = df[feature].replace([np.inf,-np.inf],np.nan)
    df[feature] = df[feature].fillna(0)
    df[feature] = pd.to_numeric(df[feature],errors='coerce')
    #Split conitnous variables into max 10 bins based on percentile
    df[feature+'_bin'] = pd.qcut(df[feature],q=10,duplicates='drop')
    for i in range(df[feature+'_bin'].nunique()):
        val = list(df[feature+'_bin'].unique())[i]
        lst.append([feature,
                   val,
                   df[df[feature+'_bin']==val].count()[feature],
                   df[(df[feature+'_bin']==val) & (df[target]=='Good')].count()[feature],
                   df[(df[feature+'_bin']==val) & (df[target]=='Bad')].count()[feature]])
    data = pd.DataFrame(lst,columns=['Variable','Value','All','Goods','Bads'])
    data['Share'] = data['All'] / data['All'].sum()
    data['Bad Rate'] = data['Bads'] / data['All']
    data['Distribution Good'] = (data['All'] - data['Bads'])/(data['All'].sum() - data['Bads'].sum())
    data['Distribution Bad'] = data['Bads']/data['Bads'].sum()
    data['WoE']=np.log(data['Distribution Good']/data['Distribution Bad'])
    data = data.replace({'WoE':{np.inf:0,-np.inf:0}})
    data['IV_bin']=data['WoE']*(data['Distribution Good'] - data['Distribution Bad'])
    data = data.sort_values(by = ['Variable','Value'],ascending = [True,True])
    data.index = range(len(data.index))
    data['IV']=data['IV_bin'].sum()
    
    return data

def calc_iv_cat(df2,feature, target):
    df=df2.copy()
    print(feature)
    lst = []
    # Ignore features having more than 100 unique values
    if df[feature].nunique()>100:
        return pd.DataFrame()
    df[feature] = df[feature].fillna(0)
    df[feature+'_bin'] = df[feature]
    for i in range(df[feature+'_bin'].nunique()):
        val = list(df[feature+'_bin'].unique())[i]
        lst.append([feature,
                   val,
                   df[df[feature+'_bin']==val].count()[feature],
                   df[(df[feature+'_bin']==val) & (df[target]=='Good')].count()[feature],
                   df[(df[feature+'_bin']==val) & (df[target]=='Bad')].count()[feature]])
    data = pd.DataFrame(lst,columns=['Variable','Value','All','Goods','Bads'])
    data['Share'] = data['All'] / data['All'].sum()
    data['Bad Rate'] = data['Bads'] / data['All']
    data['Distribution Good'] = (data['All'] - data['Bads'])/(data['All'].sum() - data['Bads'].sum())
    data['Distribution Bad'] = data['Bads']/data['Bads'].sum()
    data['WoE']=np.log(data['Distribution Good']/data['Distribution Bad'])
    data = data.replace({'WoE':{np.inf:0,-np.inf:0}})
    data['IV_bin']=data['WoE']*(data['Distribution Good'] - data['Distribution Bad'])
    #data = data.sort_values(by = ['Variable','Value'],ascending = [True,True])
    data.index = range(len(data.index))
    data['IV']=data['IV_bin'].sum()
    
    return data


def iterate_vars_iv(df,target):
    data = pd.DataFrame()
#     df=df[,2:]
    varlist = list(df.columns)
   # varlist = varlist[2:]
    for i in varlist:
        if df[str(i)].dtype == 'int' or  df[str(i)].dtype == 'float':
            data = pd.concat([data,calc_iv_num(df,str(i),target)])
        else:
            data = pd.concat([data,calc_iv_cat(df,str(i),target)])
    return data
            
    
df['target_final'] = np.where(df.bank_otput == 0, "Bad","Good")
iv_output = iterate_vars_iv(df,'target_final')



Id
AppsFlyer ID1
Client Code1
Email
Last Name
Mobile
Lead Owner Name
Lead Source
Lead Status
Lead Owner
Created Time
Modified By
Modified Time
Last Activity Time
Lead Product
Lead Sub Status
Campaign
Disposition
Phone
Referral Code
Client Code
utm_campaign
utm_content
utm_source
utm_term
utm_medium
vt_keyword
Permanent Address Proof
Correspondence Address Proof
PAN Card
Photograph
Income Proof
Bank Account Proof
eSign Eligibility
KYC Option
Photograph Approval
Income Proof Approval
Bank Account Proof Approval
Address Approval
PAN Card Approval
eSign Done
Address Proof Rejection
Bank Account Proof Rejection
Photograph Rejection
PAN Card Rejection
Income Proof Rejection
Signature Rejection
Signature Proof
Signature Proof Approval
City
PAN Number
Registration Source
Keyword
Lead Source Category
OS Type
Pickup Pincode
Partner Client Code
Total Calls
Flag1
Flag2
DNC Customer
Gender
Date of Birth
Age
AppsFlyer ID
Old-lead Owner
Rand
Lead Sub Status Temp
Appsource
AppsFlyer Channel
App Releas

In [7]:
iv_output.to_csv('IV_Output_bank.csv')